In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 4: Passport Processing ---</h2><p>You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.</p>
<p>It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.</p>
<p>Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.</p>
<p>The automatic passport scanners are slow because they're having trouble <em>detecting which passports have all required fields</em>. The expected fields are as follows:</p>
<ul>
<li><code>byr</code> (Birth Year)</li>
<li><code>iyr</code> (Issue Year)</li>
<li><code>eyr</code> (Expiration Year)</li>
<li><code>hgt</code> (Height)</li>
<li><code>hcl</code> (Hair Color)</li>
<li><code>ecl</code> (Eye Color)</li>
<li><code>pid</code> (Passport ID)</li>
<li><code>cid</code> (Country ID)</li>
</ul>
<p>Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of <code>key:value</code> pairs separated by spaces or newlines. Passports are separated by blank lines.</p>
<p>Here is an example batch file containing four passports:</p>
<pre><code>ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
</code></pre>

<p>The first passport is <em>valid</em> - all eight fields are present. The second passport is <em>invalid</em> - it is missing <code>hgt</code> (the Height field).</p>
<p>The third passport is interesting; the <em>only missing field</em> is <code>cid</code>, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing <code>cid</code> fields.  Treat this "passport" as <em>valid</em>.</p>
<p>The fourth passport is missing two fields, <code>cid</code> and <code>byr</code>. Missing <code>cid</code> is fine, but missing any other field is not, so this passport is <em>invalid</em>.</p>
<p>According to the above rules, your improved system would report <code><em>2</em></code> valid passports.</p>
<p>Count the number of <em>valid</em> passports - those that have all required fields. Treat <code>cid</code> as optional. <em>In your batch file, how many passports are valid?</em></p>
</article>


In [2]:
example = """
    ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
    byr:1937 iyr:2017 cid:147 hgt:183cm

    iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
    hcl:#cfa07d byr:1929

    hcl:#ae17e1 iyr:2013
    eyr:2024
    ecl:brn pid:760753108 byr:1931
    hgt:179cm

    hcl:#cfa07d eyr:2025 pid:166559648
    iyr:2011 ecl:brn hgt:59in
"""

required_fields = {"byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid"}  # , "cid"}
blank_line_regex = re.compile(r"(?:\r?\n){2,}")


def parse(s: str) -> list[set[str]]:
    return [
        {f.split(":")[0] for f in re.split(r"\s+", l.strip())}
        for l in re.split(blank_line_regex, s.strip())
    ]


def validate(p: set[str]) -> bool:
    return not required_fields - p


def count_valid_passports(s) -> int:
    return sum(1 for p in parse(s) if validate(p))


assert count_valid_passports(example) == 2

In [3]:
with open("../input/day4.txt") as f:
    puzzle = f.read()

print(f"Part I: {count_valid_passports(puzzle)}")

Part I: 216


<link href="style.css" rel="stylesheet"></link>
<main class="read-aloud">

<p>Your puzzle answer was <code>216</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!</p>
<p>You can continue to ignore the <code>cid</code> field, but each other field has <span title="GLORY TO ARSTOTZKA">strict rules</span> about what values are valid for automatic validation:</p>
<ul>
<li><code>byr</code> (Birth Year) - four digits; at least <code>1920</code> and at most <code>2002</code>.</li>
<li><code>iyr</code> (Issue Year) - four digits; at least <code>2010</code> and at most <code>2020</code>.</li>
<li><code>eyr</code> (Expiration Year) - four digits; at least <code>2020</code> and at most <code>2030</code>.</li>
<li><code>hgt</code> (Height) - a number followed by either <code>cm</code> or <code>in</code>:
  <ul>
  <li>If <code>cm</code>, the number must be at least <code>150</code> and at most <code>193</code>.</li>
  <li>If <code>in</code>, the number must be at least <code>59</code> and at most <code>76</code>.</li>
  </ul>
</li>
<li><code>hcl</code> (Hair Color) - a <code>#</code> followed by exactly six characters <code>0</code>-<code>9</code> or <code>a</code>-<code>f</code>.</li>
<li><code>ecl</code> (Eye Color) - exactly one of: <code>amb</code> <code>blu</code> <code>brn</code> <code>gry</code> <code>grn</code> <code>hzl</code> <code>oth</code>.</li>
<li><code>pid</code> (Passport ID) - a nine-digit number, including leading zeroes.</li>
<li><code>cid</code> (Country ID) - ignored, missing or not.</li>
</ul>
<p>Your job is to count the passports where all required fields are both <em>present</em> and <em>valid</em> according to the above rules. Here are some example values:</p>
<pre><code>byr valid:   2002
byr invalid: 2003

hgt valid: 60in
hgt valid: 190cm
hgt invalid: 190in
hgt invalid: 190

hcl valid: #123abc
hcl invalid: #123abz
hcl invalid: 123abc

ecl valid: brn
ecl invalid: wat

pid valid: 000000001
pid invalid: 0123456789
</code></pre>

<p>Here are some invalid passports:</p>
<pre><code>eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
</code></pre>

<p>Here are some valid passports:</p>
<pre><code>pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
</code></pre>

<p>Count the number of <em>valid</em> passports - those that have all required fields <em>and valid values</em>. Continue to treat <code>cid</code> as optional. <em>In your batch file, how many passports are valid?</em></p>
</article>

</main>


In [4]:
def parse(s: str) -> list[set[str]]:
    return [
        dict(f.split(":") for f in re.split(r"\s+", l.strip()))
        for l in re.split(blank_line_regex, s.strip())
    ]


def validate(passport: set[str]) -> bool:
    if required_fields - set(passport):
        return False
    # byr (Birth Year) - four digits; at least 1920 and at most 2002.
    if not 1920 <= int(passport["byr"]) <= 2002:
        return False
    # iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    if not 2010 <= int(passport["iyr"]) <= 2020:
        return False
    # eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
    if not 2020 <= int(passport["eyr"]) <= 2030:
        return False
    # hgt (Height) - a number followed by either cm or in:
    height, height_dim = passport["hgt"][:-2], passport["hgt"][-2:]
    if not height.isdecimal() or not height_dim in ("cm", "in"):
        return False
    # If cm, the number must be at least 150 and at most 193.
    if height_dim == "cm" and not 150 <= int(height) <= 193:
        return False
    # If in, the number must be at least 59 and at most 76.
    if height_dim == "in" and not 59 <= int(height) <= 76:
        return False
    # hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    if not re.match(r"^#([0-9]|[a-f]){6}$", passport["hcl"]):
        return False
    # ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    if passport["ecl"] not in ("amb", "blu", "brn", "gry", "grn", "hzl", "oth"):
        return False
    # pid (Passport ID) - a nine-digit number, including leading zeroes.
    if not re.match(r"^[0-9]{9}$", passport["pid"]):  # from start till end!
        return False
    # cid (Country ID) - ignored, missing or not.
    return True


invalid_example = """
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
"""

assert count_valid_passports(invalid_example) == 0

valid_example = """
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
"""

assert count_valid_passports(valid_example) == 4


error_pid_longer_than_9_allowed = """
byr:1977 hcl:#602927 cid:175 iyr:2010
pid:9391986394 hgt:65in eyr:2026
ecl:amb
"""

assert count_valid_passports(error_pid_longer_than_9_allowed) == 0

In [5]:
print(f"Part II: {count_valid_passports(puzzle)}")  # 151 is too high

Part II: 150


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>150</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
